In [ ]:
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt
import contextily

from pysal.viz import splot
from splot.esda import plot_moran
from pysal.explore import esda
from pysal.lib import weights
from numpy.random import seed

from esda.moran import Moran
from splot.esda import moran_scatterplot

%config InlineBackend.figure_format = "retina"

In [ ]:
# Display all columns pandas
pd.set_option("display.max_columns", None)

In [ ]:
# import sys
# import csv

# csv.field_size_limit(sys.maxsize)

In [ ]:
ms = pd.read_csv("MS_Score.csv")
ms_geo = pd.read_csv("MS.csv")

In [ ]:
df = pd.merge(ms, ms_geo.loc[:,["Cod Mun", "geometry", "centroides"]], on="Cod Mun")
print(df.shape)

In [ ]:
df.dtypes

In [ ]:
df.columns

In [ ]:
from shapely import wkt
gdf = gpd.GeoDataFrame(data=df)
gdf["geometry"] = gdf["geometry"].apply(wkt.loads)
gdf.set_geometry("geometry")
gdf = gdf.set_crs(crs="epsg:4674").to_crs(epsg=5880)

In [ ]:
gdf.crs

In [ ]:
gdf.head()

In [ ]:
fig, ax = plt.subplots(1, figsize=(10, 10))
gdf.plot(
    column="Score",
    cmap="viridis",
    scheme="quantiles",
    k=5,
    edgecolor="white",
    linewidth=0.0,
    alpha=0.75,
    legend=True,
    legend_kwds={"loc":2},
    ax=ax)
contextily.add_basemap(ax, crs=gdf.crs)
plt.show()

## Moran

In [ ]:
w = weights.KNN.from_dataframe(gdf, k=8) #get 8 neighbor
w.transform = "R" #normalize all the rows

In [ ]:
moran = Moran(gdf["Score"], w)

In [ ]:
moran.I

In [ ]:
fig, ax = moran_scatterplot(moran, aspect_equal = True)
plt.show()

## Lisa

In [ ]:
from esda.moran import Moran_Local
from splot.esda import lisa_cluster 
from splot.esda import plot_local_autocorrelation

In [ ]:
moran_loc = Moran_Local(gdf["Score"], w, )

In [ ]:
gdf["moran_p_value"] = moran_loc.p_sim

In [ ]:
# Get LISA clusters
clusters = lisa_cluster(moran_loc, gdf, p=0.05)

In [ ]:
plot_local_autocorrelation(moran_loc, gdf, "Score")
plt.show()

In [ ]:
# Add moran to columns
gdf["moran"] = moran_loc.Is

In [ ]:
gdf["cluster_label"] = moran_loc.q

# Create a dictionary to map cluster labels to colors
cluster_colors = {1: "red", 2: "#e28743", 3: "#23dcc8", 4: "blue", 5: "grey"}

# Create a dictionary to map cluster labels to labels
cluster_labels = {1: "High-High", 2: "Low-High", 3: "Low-Low", 4: "High-Low", 5: "Not Significant"}

# Set cluster_label to 5 when moran_p_value is less than 0.05
gdf.loc[gdf["moran_p_value"] > 0.05, "cluster_label"] = 5

# Map the colors to the cluster labels
gdf["cluster_color"] = gdf["cluster_label"].map(cluster_colors)

# Map the labels to the cluster labels
gdf["cluster_label"] = gdf["cluster_label"].map(cluster_labels)


In [ ]:
# Drop all columns except geometry and moran and Cod Mun and Municipios and Score
gdf = gdf.loc[:,["geometry", "moran", "Cod Mun", "Municípios", "Score", "cluster_label", "cluster_color", "centroides", "moran_p_value"]]

In [ ]:
gdf.explore(
    column="moran",
    tooltip=["Municípios"],
    popup=True,
    cmap="Blues",
    tiles="Stamen Terrain",
    scheme="quantiles",
    k=20

)

In [ ]:
ls_color = cluster_colors.values()

In [ ]:
import matplotlib.colors as colors
gdf.explore(
    column="cluster_label",
    # tooltip=["Municípios"],
    popup=True,
    cmap=colors.ListedColormap(ls_color),
    tiles="Stamen Terrain",
    # scheme="quantiles",
    k=20

)